In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from utils.data import *
from transform import *
from dataset import PlanktonDataset

In [3]:
data_transform = {
    'train': transforms.Compose([
        Rescale((224, 224)),
        RandomHorizontalFlip(),
        RandomVerticalFlip(),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        Rescale((224, 224)),
        ToTensor(),
        Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

img_dir = 'data/images'
csv_filename = 'data/head_tail_results.csv'


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epoch=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    
    for epoch in range(num_epoch):
        print('Epoch{}/{}'.format(epoch, num_epoch - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)
                
            running_loss = 0.0
            
            for data in dataloaders[phase]:
                inputs, target = data['image'], data['coordinates']
                